In [15]:
import os
import glob
from openai import OpenAI
import gradio as gr
from dotenv import load_dotenv
from IPython.display import Markdown, display

In [2]:
from openai import api_key

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')
openai = OpenAI(api_key=api_key)

In [3]:
MODEL = 'gpt-4o-mini'

In [4]:
context = {}

employees = glob.glob("knowledge-base/employees/*")
for employee in employees:
    name = employee.split(' ')[-1][:-3]
    info = ""
    with open(employee, 'r', encoding='utf-8') as f:
        info = f.read()
    context[name] = info


In [5]:
products = glob.glob("knowledge-base/products/*")
for product in products:
    name = product.split(os.sep)[-1][:-3]
    doc = ""
    with open(product, "r", encoding='utf-8') as f:
        doc = f.read()
    context[name] = doc

In [28]:
# print(context['Carllm'])

In [29]:
# print(context['Lancaster'])

In [30]:
# context.keys()

In [9]:
def get_relevant_context(message):
    relevant_message = []
    for context_title, context_details in context.items():
        if context_title.lower() in message.lower():
            relevant_message.append(context_details)
    return relevant_message

In [31]:
# get_relevant_context("Who is lancaster?")

In [ ]:
system_message = "You are an expert in answering accurate questions about Insurellm, the Insurance Tech company. Give brief, accurate answers. If you don't know the answer, say so. Do not make anything up if you haven't been provided with relevant context."

In [19]:
def add_context(message):
    relevant_context = get_relevant_context(message)
    if relevant_context:
        message += "\n\nThe following additional context might be relevant in answering this question:\n\n"
        for info in relevant_context:
            message += info + "\n\n"
    return message

In [24]:
# print(add_context("Who is lancaster?"))

In [26]:
def chat(message, history):
    messages = [{'role':'system', 'content': system_message}] + history
    user_message = add_context(message)
    messages.append({'role':'user', 'content': user_message})
    stream = openai.chat.completions.create(model=MODEL, messages=messages, stream=True)
    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield response

In [27]:
view = gr.ChatInterface(chat, type="messages").launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.
